Part 1

In [3]:
import pandas as pd
import re
import numpy as np
import copy

In [4]:
def str_int(s):
    return list(map(int, s.strip().split(' ')))

In [5]:
# file = "test.dat"
file = "day.dat"
lines = open(file,'r').read().split('\n')
seeds = pd.DataFrame({'seed':str_int(lines[0].split(':')[1])})
map_types = ['seed-to-soil map:','soil-to-fertilizer map:',
             'fertilizer-to-water map:','water-to-light map:',
             'light-to-temperature map:','temperature-to-humidity map:',
             'humidity-to-location map:']

df = pd.DataFrame(columns=['map_from','map_to','destination','source','length'])
for l in lines[1:]:
    if l in map_types:
        match = re.match(r'^(\w+)\-to\-(\w+)', l)
        map_from, map_to = match.group(1), match.group(2)
    elif l != '':
        dest,src,length = str_int(l)
        df = pd.concat([df,pd.DataFrame({'map_from':map_from, 'map_to':map_to,
                                         'destination':dest, 'source':src,
                                         'length':length}, index=[0])])
df = df.reset_index(drop=True)   
df.head()

,map_from,map_to,destination,source,length
0,seed,soil,2211745924,1281207339,39747980
1,seed,soil,3648083739,2564129012,145170114
2,seed,soil,4171944574,2333022880,44675857
3,seed,soil,540694760,848661020,78793182
4,seed,soil,256996824,588160543,260500477


In [6]:
seeds.head()

,seed
0,3082872446
1,316680412
2,2769223903
3,74043323
4,4131958457


In [7]:
from_col = 'seed'
while from_col != 'location':
    map_list = df[df['map_from'] == from_col]
    to_col = list(map_list['map_to'])[0]
    seeds[to_col] = seeds[from_col]
    for i, s in enumerate(seeds[from_col]):
        for _, m in map_list.iterrows():
            if m['source'] <= s < m['source'] + m['length']:
                seeds.at[i, to_col] = m['destination'] + s - m['source']
    from_col = to_col

seeds.head()

,seed,soil,fertilizer,water,light,temperature,humidity,location
0,3082872446,3146600596,3070093545,1126696133,2291994928,1972007158,1311845980,1085436433
1,316680412,529355900,2791332414,847935002,1268090341,64848467,84470622,84470622
2,2769223903,4276545208,4187659549,2037385377,2943142680,1678956429,1018795251,797022648
3,74043323,2119458269,1459707245,4064254365,3341541755,2961582857,4109588370,3040934752
4,4131958457,2994950048,2994950048,1051552636,2216851431,2473940599,2473940599,2473940599


In [8]:
#answer is 84470622
seeds['location'].min()

84470622

Part 2

In [9]:
file = "test.dat"
# file = "day.dat"
lines = open(file,'r').read().split('\n')
raw_seeds = str_int(lines[0].split(':')[1])
seeds = pd.DataFrame({'source':raw_seeds[::2], 'length':raw_seeds[1::2]})
map_types = ['seed-to-soil map:','soil-to-fertilizer map:',
             'fertilizer-to-water map:','water-to-light map:',
             'light-to-temperature map:','temperature-to-humidity map:',
             'humidity-to-location map:']
df = pd.DataFrame(columns=['map_from','map_to','destination','source','length'])
for l in lines[1:]:
    if l in map_types:
        match = re.match(r'^(\w+)\-to\-(\w+)', l)
        map_from, map_to = match.group(1), match.group(2)
    elif l != '':
        dest,src,length = str_int(l)
        df = pd.concat([df,pd.DataFrame({'map_from':map_from, 'map_to':map_to,
                                         'destination':dest, 'source':src,
                                         'length':length}, index=[0])])
df = df.reset_index(drop=True)   
df

,map_from,map_to,destination,source,length
0,seed,soil,50,98,2
1,seed,soil,52,50,48
2,soil,fertilizer,0,15,37
3,soil,fertilizer,37,52,2
4,soil,fertilizer,39,0,15
5,fertilizer,water,49,53,8
6,fertilizer,water,0,11,42
7,fertilizer,water,42,0,7
8,fertilizer,water,57,7,4
9,water,light,88,18,7


For the temperature->humidity map the output endpoints in the humidity domain are 
[0, 55, 56, 92, 93, 96, 97, inf] (computed from the humidity->location map inversion)  
the map (0 69 1 / 1 0 69) is defined by f(x) = x + 1 if 0 <= x < 69, 0 if x == 69, x if x > 69  
the map endpoints in the temp domain are 
[0, 68, 69, 70, inf]   

if we map the output endpoints backward through the map, we get the corresponding (sorted) inputs in the temp domain   
[*54*, 55, 69, 92, 93, 96, 97, inf] (so the 54 comes about by solving x+1 = 55 from the map definition)  
the union of the above endpoints is [0, 54, 55, 68, 69, 70, 92, 93, 96, 97, inf]

In [27]:
# let's get the endpoints
map_names = list(pd.concat([df['map_from'],df['map_to']]).unique())[-2::-1]
range_list = [0,np.inf]
for m in map_names:
    range_vals = df[df['map_from'] == m][['destination','source','length']].sort_values(by='source')
    new_range_list = []
    for _, row in range_vals.iterrows():
        a, b, c = row['source'], row['source']+row['length'], row['destination']
        new_range_list.extend([a,b,a-1,b-1])
        print(m, a,b,c, new_range_list)
    new_range_list = sorted(list(set(new_range_list)))
    print(m, new_range_list)
#     range_list = sorted(list(set(range_list)))
#     print(m, range_list)
# range_vals = seeds[['source','length']].sort_values(by='source')
# for index, row in range_vals.iterrows():
#     a = row['source']
#     b = row['source']+row['length']
#     range_list.extend([a,b,a-1,b-1,a+1,b+1])
# range_list = sorted(list(set(range_list)))
# print(range_list)


humidity 56 93 60 [56, 93, 55, 92]
humidity 93 97 56 [56, 93, 55, 92, 93, 97, 92, 96]
humidity [55, 56, 92, 93, 96, 97]
temperature 0 69 1 [0, 69, -1, 68]
temperature 69 70 0 [0, 69, -1, 68, 69, 70, 68, 69]
temperature [-1, 0, 68, 69, 70]
light 45 64 81 [45, 64, 44, 63]
light 64 77 68 [45, 64, 44, 63, 64, 77, 63, 76]
light 77 100 45 [45, 64, 44, 63, 64, 77, 63, 76, 77, 100, 76, 99]
light [44, 45, 63, 64, 76, 77, 99, 100]
water 18 25 88 [18, 25, 17, 24]
water 25 95 18 [18, 25, 17, 24, 25, 95, 24, 94]
water [17, 18, 24, 25, 94, 95]
fertilizer 0 7 42 [0, 7, -1, 6]
fertilizer 7 11 57 [0, 7, -1, 6, 7, 11, 6, 10]
fertilizer 11 53 0 [0, 7, -1, 6, 7, 11, 6, 10, 11, 53, 10, 52]
fertilizer 53 61 49 [0, 7, -1, 6, 7, 11, 6, 10, 11, 53, 10, 52, 53, 61, 52, 60]
fertilizer [-1, 0, 6, 7, 10, 11, 52, 53, 60, 61]
soil 0 15 39 [0, 15, -1, 14]
soil 15 52 0 [0, 15, -1, 14, 15, 52, 14, 51]
soil 52 54 37 [0, 15, -1, 14, 15, 52, 14, 51, 52, 54, 51, 53]
soil [-1, 0, 14, 15, 51, 52, 53, 54]
seed 50 98 52 [50, 9

In [10]:
seeds

,source,length
0,79,14
1,55,13


seed number 82  
soil 84  
fertilizer 84   
water 84   
light 77  
temperature 45   
humidity 46  
location 46  

In [ ]:
# map_names = pd.concat([df['map_from'],df['map_to']]).unique()
# best = {key: np.inf for key in map_names}
# current = {key: np.inf for key in map_names}

# i=0
# for s_start, s_end in zip(seeds['seed_start'], seeds['seed_range']):
#     i+=1
#     print(i)
#     seed_range = range(s_start, s_start+s_end)
#     for s in seed_range:
#         current['seed'] = s
#         from_col = 'seed'
#         while from_col != 'location':
#             map_list = df[df['map_from'] == from_col]
#             to_col = list(map_list['map_to'])[0]
#             current[to_col] = current[from_col]
#             # if s == 82:
#             #     print(from_col, to_col)
#             #     print(map_list)
#             for _, m in map_list.iterrows():
#                 if m['source'] <= s < m['source'] + m['length']:
#                     current[to_col] = m['destination'] + s - m['source']
#                     # if s == 82:
#                     #     print("hit", from_col, to_col, m['destination'], s, m['source'], m['length'])
#             from_col = to_col
#             if current['location'] < best['location']: 
#                 best = copy.deepcopy(current)
#         # if s == 82:
#         #     print(current)
# best